In [3]:
from gensim import corpora, models, similarities
import gensim

In [4]:
import json 
import gzip 
#from StringIO import StringIO
import urllib.request
import requests, zipfile, io
import re

def parse(url): 
    if re.match("^http", url):
        r = requests.get(url)
        g = gzip.open(io.BytesIO(r.content))
    else:
        g = gzip.open(url, 'r') 
    for l in g:
        yield eval(l)

import json 
import gzip 
def parse(url): 
    g = gzip.open(url, 'r') 
    for l in g: 
        yield eval(l)

In [5]:
import gensim
import nltk
stoplist = nltk.corpus.stopwords.words('english')
lemma = nltk.stem.wordnet.WordNetLemmatizer()

def myReviews(review_files_urls, catFilter = None, asin = None, concatenate = False):
    preprocess_reviewText = []
    for url in review_files_urls:
        print("Processing url: " + url)
        review_item_iterator = iter(parse(url))
        it = 0
        conc_count = 0
        for review_item in review_item_iterator:
            it += 1
        #for it in range(1000000):
            if it%100000==0:
                print("it="+str(it))

            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)
            preprocess_reviewText_i = [lemma.lemmatize(word) for word in preprocess_reviewText_i if word not in stoplist]


            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
                if concatenate==False:
                    yield preprocess_reviewText_i
                else:
                    conc_count += 1
                    preprocess_reviewText.extend(preprocess_reviewText_i)
                    print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

In [7]:
#base_url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/"
base_url = "/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/"

from bs4 import BeautifulSoup
import urllib.request


html = urllib.request.urlopen("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/")
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())
all_html_links = soup.find_all('a')


from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for links in all_html_links:
    if re.match("^review.*[^_5|_10].json.gz$", links["href"]):
        print(links["href"])

In [8]:
review_file_names = [links["href"] for links in all_html_links if re.match("^review.*[^_5|_10].json.gz$", links["href"])]

review_file_names= ["reviews_Books_10.json.gz" if file_name =="reviews_Books.json.gz" else file_name for file_name in review_file_names]
print(review_file_names)
review_file_urls = [base_url + review_file_name for review_file_name in review_file_names]
#print(review_file_urls)

['reviews_All_Beauty.json.gz', 'reviews_All_Credit_Cards.json.gz', 'reviews_All_Electronics.json.gz', 'reviews_Alternative_Rock.json.gz', 'reviews_Amazon_Coins.json.gz', 'reviews_Amazon_Fashion.json.gz', 'reviews_Amazon_Fire_TV.json.gz', 'reviews_Amazon_Instant_Video.json.gz', 'reviews_Appliances.json.gz', 'reviews_Apps_for_Android.json.gz', 'reviews_Appstore_for_Android.json.gz', 'reviews_Arts_Crafts_and_Sewing.json.gz', 'reviews_Automotive.json.gz', 'reviews_Baby.json.gz', 'reviews_Baby_Products.json.gz', 'reviews_Beauty.json.gz', 'reviews_Blues.json.gz', 'reviews_Books_10.json.gz', 'reviews_Broadway_and_Vocalists.json.gz', 'reviews_Buy_a_Kindle.json.gz', 'reviews_CDs_and_Vinyl.json.gz', 'reviews_Camera_and_Photo.json.gz', 'reviews_Car_Electronics.json.gz', 'reviews_Celebrate_your_Birthday_with_Nickelodeon.json.gz', 'reviews_Cell_Phones_and_Accessories.json.gz', "reviews_Children's_Music.json.gz", 'reviews_Christian.json.gz', 'reviews_Classic_Rock.json.gz', 'reviews_Classical.json.gz

In [9]:
  
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary()
    for review_file_url in review_file_urls:
        dictionary.add_documents(myReviews([review_file_url]))
        from six import iteritems
        once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
        dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
        dictionary.compactify()  # remove gaps in id sequence after words that were removed
        print(dictionary)
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)



Processing url: /media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_All_Beauty.json.gz
Dictionary(14065 unique tokens: ['like', 'sticker', 'hassle', 'keep', 'item']...)
Processing url: /media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_All_Credit_Cards.json.gz
Dictionary(15365 unique tokens: ['like', 'sticker', 'hassle', 'keep', 'item']...)
Processing url: /media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_All_Electronics.json.gz
Dictionary(28193 unique tokens: ['like', 'sticker', 'hassle', 'keep', 'item']...)
Processing url: /media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_Alternative_Rock.json.gz
Dictionary(31785 unique tokens: ['like', 'sticker', 'hassle', 'keep', 'item']...)
Processing url: /media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_Amazon_Coins.json.gz
Dictionary(31794 unique tokens: ['like', 'sticker', 'hassle', 'keep', 'item']...)
Processing url: /media/mister/ntfs/Rainer/y2bu

FileNotFoundError: [Errno 2] No such file or directory: '/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/reviews_Books_10.json.gz'


    
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary(myReviews(review_file_urls))
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

In [ ]:
def myCorpus(review_file_urls, catFilter = None, asin = None, concatenate = False):
    for review in myReviews(review_file_urls,catFilter,asin,concatenate):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [ ]:
import os
tfidfFileName ='../../download/word2vec/reviews_all_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    #tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf = models.TfidfModel(myCorpus(review_file_urls))
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

In [ ]:
import pickle

tfidf_document_FileName ='../../download/word2vec/reviews_cell_phones_review.tfidf'
if os.path.isfile(tfidf_document_FileName)==False:
    document = next(myCorpus(review_file_urls = ['../../download/word2vec/reviews_Cell_Phones_and_Accessories.json.gz'], catFilter="Cell Phones",concatenate=True))
    document_tfidf = tfidf[document]  
    pickle.dump(document_tfidf, open(tfidf_document_FileName, 'wb'))
else:
    document_tfidf = pickle.load( open( tfidf_document_FileName, "rb" ) )
    
    


In [ ]:
print([(dictionary.get(tokenid),tfidf_value) for (tokenid,tfidf_value) in document_tfidf if tfidf_value >0.1])

In [ ]:
print(tfidf)